In [1]:
from IPython.display import clear_output
%cd /content/drive/MyDrive/collab_sandbox/learn_DL/dive_into_dl/
# !pip install colabcode
clear_output()

In [14]:
import pandas as pd
import torch
import numpy as np
from torch.utils import data
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

import matplotlib.pyplot as plt
import seaborn.apionly as sns
from sklearn.model_selection import KFold
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
dataset = pd.read_csv("./data/HousePricesKaggle/train.csv")

drop_columns = 'Id'.split()
dataset = dataset.drop(columns=drop_columns)

select_columns = dataset.dtypes[dataset.dtypes != 'object'].index.tolist()
select_columns = list(set(select_columns) - set(['SalePrice']))

dataset[select_columns] = dataset[select_columns].apply(lambda x: (x - x.mean()) / (x.std()))

dataset[select_columns] = dataset[select_columns].fillna(dataset[select_columns].mean() / dataset[select_columns].std())

# encode to one hot https://stackoverflow.com/questions/58101126/using-scikit-learn-onehotencoder-with-a-pandas-dataframe
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)

object_columns = dataset.dtypes[dataset.dtypes == 'object'].index.tolist()
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(dataset[object_columns]))

OH_cols_train.index = dataset.index
num_X_train = dataset.drop(object_columns, axis=1)

dataset = pd.concat([num_X_train, OH_cols_train], axis=1)

dataset.head()


,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice,0,1,2,...,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267
0,0.073350,-0.207948,-0.207071,0.651256,-0.517023,1.050634,0.878367,0.509840,0.575228,-0.288554,-0.944267,-0.459145,-0.793162,1.161454,-0.120201,0.370207,1.107431,-0.240978,0.789470,1.227165,0.163723,-0.211381,0.911897,-0.950901,0.992066,0.311618,0.350880,-0.751918,0.216429,-0.359202,-0.116299,-0.270116,-0.068668,-0.087658,-1.598563,0.138730,208500,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,-0.872264,0.409724,-0.091855,-0.071812,2.178881,0.156680,-0.429430,-0.572637,1.171591,-0.288554,-0.641008,0.466305,0.257052,-0.794891,-0.120201,-0.482347,-0.819684,3.947457,0.789470,-0.761360,0.163723,-0.211381,-0.318574,0.600289,-0.101506,0.311618,-0.060710,1.625638,-0.704242,-0.359202,-0.116299,-0.270116,-0.068668,-0.087658,-0.488943,-0.614228,181500,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.073350,-0.084413,0.073455,0.651256,-0.517023,0.984415,0.829930,0.322063,0.092875,-0.288554,-0.301540,-0.313261,-0.627611,1.188943,-0.120201,0.514836,1.107431,-0.240978,0.789470,1.227165,0.163723,-0.211381,-0.318574,0.600289,0.911061,0.311618,0.631510,-0.751918,-0.070337,-0.359202,-0.116299,-0.270116,-0.068668,-0.087658,0.990552,0.138730,223500,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.309753,-0.413838,-0.096864,0.651256,-0.517023,-1.862993,-0.720051,-0.572637,-0.499103,-0.288554,-0.061648,-0.687089,-0.521555,0.936955,-0.120201,0.383528,1.107431,-0.240978,-1.025689,-0.761360,0.163723,-0.211381,0.296662,0.600289,0.789553,1.649742,0.790533,-0.751918,-0.175988,4.091122,-0.116299,-0.270116,-0.068668,-0.087658,-1.598563,-1.367186,140000,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
4,0.073350,0.574436,0.375020,1.374324,-0.517023,0.951306,0.733056,1.360357,0.463410,-0.288554,-0.174805,0.199611,-0.045596,1.617323,-0.120201,1.298881,1.107431,-0.240978,0.789470,1.227165,1.389547,-0.211381,1.527133,0.600289,0.870558,1.649742,1.697903,0.779930,0.563567,-0.359202,-0.116299,-0.270116,-0.068668,-0.087658,2.100173,0.138730,250000,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


In [ ]:
# fig, ax = plt.subplots(figsize=(40,40))
# corr = dataset_train.corr()# plot the heatmap
# sns.heatmap(corr, xticklabels=corr.columns, yticklabels=corr.columns, annot=True, cmap=sns.diverging_palette(220, 20, as_cmap=True), ax=ax)

In [ ]:
# def encode_categorical(dataset, columns, full_dataset):
#   dataset = dataset.copy(deep=True)
#   # print(dataset)
#   dataset_cols = dataset.columns.values
#   for column in columns:
#     if column in dataset_cols:
#       values_list = full_dataset[column].values.tolist()
#       list_columns = sorted(list(map(str, list(set(values_list)))))
#       list_columns_renamed = [f'{column}_{item}' for item in list_columns]
#       dict_columns = {item: i for (i, item) in enumerate(list_columns)}
      
#       # print(dataset[column].)
#       dataset_values = dataset[column].tolist()
#       for i, value in enumerate(dataset_values):
#         value = str(value)

#         ohe_hot = [0 for _ in range(len(list_columns))]
#         if dict_columns.get(value):
#           ohe_hot[dict_columns.get(value)] = 1

#         # slow operation!
#         dataset.loc[i, list_columns_renamed] = ohe_hot
#       dataset = dataset.drop(columns=[column],) 
  
#   return dataset

In [ ]:
# dataset_train.dtypes[dataset_train.dtypes == 'object'].index.tolist()

In [ ]:
# categorical_labels = dataset_train.dtypes[dataset_train.dtypes == 'object'].index.tolist() #list(set('MSSubClass PavedDrive MSZoning Street	Alley	LotShape	LandContour	Utilities	LotConfig	LandSlope	Neighborhood	Condition1	Condition2	BldgType	HouseStyle RoofStyle	RoofMatl	Exterior1st	Exterior2nd	MasVnrType ExterQual	ExterCond	Foundation	BsmtQual	BsmtCond	BsmtExposure	BsmtFinType1 BsmtFinType2 Heating	HeatingQC	CentralAir	Electrical KitchenQual Functional FireplaceQu	GarageType GarageFinish PoolQC	Fence	MiscFeature SaleType	SaleCondition GarageCond GarageQual GarageType GarageFinish	GarageQual GarageCond'.split()))
# dataset_train = encode_categorical(dataset_train, categorical_labels, dataset)
# dataset_test = encode_categorical(dataset_test, categorical_labels, dataset)
# dataset_train.head(5)

In [ ]:
# len(dataset_train.columns) == len(dataset_test.columns), len(dataset_test.columns), len(dataset_train.columns)

In [ ]:
# dataset_train.head(5)

In [ ]:
# dataset_test.head(5)

In [ ]:
# def standardize_data(dataset, exclude_columns,):
#   dataset = dataset.copy(deep=True)
#   dataset_cols = list(set(dataset.columns.values) - set(exclude_columns))
#   for column_name in dataset_cols:
#     column = dataset[column_name].to_numpy()
#     # print(column, column_name)
#     nan_indexes = np.isnan(column)
#     real_indexes = ~np.isnan(column)

#     n = real_indexes.astype(int).sum()
#     real_column = column[real_indexes]

#     real_column_mean = real_column.sum() / n # or .mean()  
#     real_column_var = (np.sum([(column[i]-real_column_mean)**2 if item else 0 for i, item in enumerate(list(real_indexes))]) / n)**(1/2)
#     std_column = np.array([(column[i]-real_column_mean) / real_column_var if item else 0 for i, item in enumerate(list(real_indexes))])
    
#     if np.isnan(std_column).any():
#       std_column = np.zeros_like(std_column)
    
#     dataset.loc[:, column_name] = std_column 
#   return dataset

# # dataset_train = standardize_data(dataset_train, ['SalePrice'])
# # dataset_test = standardize_data(dataset_test, ['SalePrice'])
# # dataset_train.head()

In [ ]:
# dataset_test.head()

In [ ]:
# fig, ax = plt.subplots(figsize=(40,40))
# corr = dataset_train.corr()# plot the heatmap
# sns.heatmap(corr, xticklabels=corr.columns, yticklabels=corr.columns, annot=True, cmap=sns.diverging_palette(220, 20, as_cmap=True), ax=ax)

In [41]:
class MultiLayerModel(nn.Module):
  def __init__(self, 
    in_features=None,
    out_features=None,
    dropout=0.1
               
    ):
    super(MultiLayerModel, self).__init__()
   
    self.lin_1 = nn.Linear(in_features=in_features, out_features=in_features, ) 
    self.lin_2 = nn.Linear(in_features=in_features, out_features=in_features, ) 
    self.lin_3 = nn.Linear(in_features=in_features, out_features=out_features, ) 

    self.relu = nn.ReLU()
    self.tanh = nn.Tanh()
    self.gelu = nn.GELU()
    self.selu = nn.SELU()

    self.drop_1 = nn.Dropout(p=dropout)
    self.drop_2 = nn.Dropout(p=dropout)


    # self.norm = nn.LayerNorm()

    self.eps = 0.0001

  def normalize(self, x):
    return (x - x.mean(dim=0))/(x.std(dim=0)+self.eps)

  def forward(self, x):
    x = self.drop_1(self.gelu(self.lin_1(x)))
    x = self.drop_1(self.selu(self.lin_2(x)))
    x = self.lin_3(x)
    return x

class RMSLELogLoss(nn.Module):
  def __init__(self):
    super().__init__()
    self.mse = nn.MSELoss()
      
  def forward(self, pred, actual):
    clipped_preds = torch.clamp(pred, 1, float('inf'))
    return torch.sqrt(self.mse(torch.log(clipped_preds), torch.log(actual)))

model = MultiLayerModel(in_features=3, out_features=1)
print(model(torch.tensor([[0.1, 0.2, 0.3], [0.4, 0.5, 0.6], [0.7, 0.8, 0.9], [0.11, 0.12, 0.13]], dtype=torch.float32, )))
predicts = model(torch.tensor([[0.1, 0.2, 0.3], [0.4, 0.5, 0.6], [0.7, 0.8, 0.9], [0.11, 0.12, 0.13]], dtype=torch.float32, ))
loss = RMSLELogLoss()(predicts, torch.tensor([[0.12], [0.15], [0.18], [0.19]], ))
print(loss)
loss.backward()

tensor([[-0.0491],
        [-0.1591],
        [ 0.2540],
        [-0.0641]], grad_fn=<AddmmBackward0>)
tensor(1.8570, grad_fn=<SqrtBackward0>)


In [ ]:
class CustomPandasDataset(Dataset):
  def __init__(self, 
                dataset=None,
                target_columns=None,
                infer=False
      ):
      self.dataset = dataset
      self.target_columns = target_columns
      self.infer = infer
      self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

  def __len__(self):
      return len(self.dataset)

  def __getitem__(self, idx):
      if self.infer:
        data_row = self.dataset.iloc[idx]
        target = torch.tensor([0], dtype=torch.float32, device=self.device)
        features = torch.tensor(data_row.values, dtype=torch.float32, device=self.device)
        return features, target 
      else:
        data_row = self.dataset.iloc[idx]
        target = torch.tensor(data_row[self.target_columns].values, dtype=torch.float32, device=self.device)
        data_row = data_row.drop(labels=self.target_columns)
        features = torch.tensor(data_row.values, dtype=torch.float32, device=self.device)

        return features, target 

temp_dataset = CustomPandasDataset(
    dataset=dataset,
    target_columns=['SalePrice'],
    # infer=True
)

temp_dataloader = DataLoader(
    dataset=temp_dataset, 
    batch_size=16, 
    shuffle=True
)

next(iter(temp_dataloader))
# temp_dataset

# Cross validation class

In [42]:
class Trainer:
  def __init__(self, 
    model=None, 
    dataset=None,
    optimizer_class=None,
    loss_func=None,
    estimate_func=None,
    batch_size=None,
    model_hyperparams=None,
    model_class=None
    ): 
    self.model = None
    self.dataset = dataset
    self.optimizer_class = optimizer_class
    self.optimizer = None
    self.loss_func = loss_func
    self.estimate_func = estimate_func
    # self.folds_models = []
    self.folds_avg_train_loss = []
    self.folds_avg_test_loss = []
    self.batch_size = batch_size

    self.model_class = model_class
    self.model_hyperparams = model_hyperparams

    self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(self.device)
    

  def train(self, 
            epochs=1,
            lr=5,
            k_folds=5,
            weight_decay=0
    ):
    # source: https://www.machinecurve.com/index.php/2021/02/03/how-to-use-k-fold-cross-validation-with-pytorch/
    
    kfold = KFold(n_splits=k_folds, shuffle=True)

    for fold_num, (train_ids, test_ids) in enumerate(kfold.split(self.dataset)):
      print(f"Fold {fold_num}")

      train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
      test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)
      
      trainloader = DataLoader(
        self.dataset, 
        batch_size=self.batch_size,
        sampler=train_subsampler
      )
      
      testloader = DataLoader(
        self.dataset,
        batch_size=self.batch_size,
        sampler=test_subsampler
      )
      
      self.model = self.model_class(**self.model_hyperparams)
      self.model.to(self.device)
      self.optimizer = self.optimizer_class(self.model.parameters(), lr=lr, weight_decay=weight_decay)
      
      fold_train = 0
      fold_test = 0
      for epoch in range(epochs):
        train_loss = 0
        test_loss = 0
        
        # train
        self.model.train()
        for features, targets in trainloader:
          self.optimizer.zero_grad()

          predicts = self.model(features)
          loss = self.loss_func(predicts, targets)
          loss.backward()
          self.optimizer.step()

          # train metric
          with torch.no_grad():
            train_loss += self.estimate_func(predicts, targets)
        
        # test metric
        self.model.eval()
        with torch.no_grad():
          for features, targets in testloader:
            predicts = self.model(features)
            test_loss += self.estimate_func(predicts, targets)
        
        avg_train = train_loss/len(trainloader)
        avg_test = test_loss/len(testloader)
        fold_train = avg_train
        fold_test  = avg_test 
        print(f"Fold {fold_num} Epoch {epoch}: train: {avg_train} test: {avg_test}")

      last_fold_train = fold_train #/ epochs
      last_fold_test = fold_test #/ epochs

      print(f"Fold {fold_num} ---- Train: {last_fold_train} Test: {last_fold_test} ---- ")

      self.folds_avg_train_loss.append(last_fold_train)
      self.folds_avg_test_loss.append(last_fold_test)

      # del self.model
      # del self.optimizer

    print("Statistics:")
    for i in range(k_folds):
      train_l = self.folds_avg_train_loss[i]
      test_l = self.folds_avg_test_loss[i]
      print(f"Fold {i}: Train: {train_l} Test: {test_l} diff {abs(train_l - test_l)}")

  def train_pred(self, 
    epochs=1,
    lr=5,
    weight_decay=0
  ):

    trainloader = DataLoader(
      dataset=self.dataset, 
      batch_size=self.batch_size
    )

    self.model = self.model_class(**self.model_hyperparams)
    self.optimizer = self.optimizer_class(self.model.parameters(), lr=lr, weight_decay=weight_decay)

    for epoch in range(epochs):
      train_loss = 0
      test_loss = 0

      # train
      for features, targets in trainloader:
        self.optimizer.zero_grad()

        predicts = self.model(features)
        loss = self.loss_func(predicts, targets)
        loss.backward()
        self.optimizer.step()

        # train metric
        with torch.no_grad():
          train_loss += self.estimate_func(predicts, targets)

      avg_train = train_loss/len(trainloader) 
      print(f"Epoch {epoch}: train: {avg_train}")
      

target_columns = ['SalePrice']
batch_size = 64

model_hyperparams = {
    'in_features': dataset.shape[1]-1,
    'out_features': 1,
    'dropout': 0.1
}

custom_dataset = CustomPandasDataset(
    dataset, 
    target_columns=target_columns,
)
rmse_loss = RMSLELogLoss()
optimizer_class = torch.optim.Adam
loss_func_1 = nn.MSELoss()
loss_func_2 = nn.L1Loss()

trainer = Trainer(
  dataset=custom_dataset,
  optimizer_class=optimizer_class,
  loss_func=loss_func_1, 
  estimate_func=rmse_loss,
  batch_size=batch_size,
  model_hyperparams=model_hyperparams,
  model_class=MultiLayerModel
)
# trainer.train(epochs=60, lr=7,)

cuda


In [43]:
trainer.train(epochs=35, lr=0.2,)

Fold 0
Fold 0 Epoch 0: train: 2.424511671066284 test: 0.21847878396511078
Fold 0 Epoch 1: train: 0.22551272809505463 test: 0.18541792035102844
Fold 0 Epoch 2: train: 0.15607497096061707 test: 0.17564958333969116
Fold 0 Epoch 3: train: 0.12160264700651169 test: 0.19827565550804138
Fold 0 Epoch 4: train: 0.12863194942474365 test: 0.19653752446174622
Fold 0 Epoch 5: train: 0.12343277782201767 test: 0.17496371269226074
Fold 0 Epoch 6: train: 0.12321896106004715 test: 0.19761201739311218
Fold 0 Epoch 7: train: 0.11861254274845123 test: 0.19256499409675598
Fold 0 Epoch 8: train: 0.11268194019794464 test: 0.18893884122371674
Fold 0 Epoch 9: train: 0.1131211444735527 test: 0.20543327927589417
Fold 0 Epoch 10: train: 0.11474820226430893 test: 0.20395179092884064
Fold 0 Epoch 11: train: 0.10980380326509476 test: 0.2063368856906891
Fold 0 Epoch 12: train: 0.11225268989801407 test: 0.1971169412136078
Fold 0 Epoch 13: train: 0.11161348968744278 test: 0.1948140263557434
Fold 0 Epoch 14: train: 0.101

In [ ]:
trainer.train_pred(epochs=20, lr=0.1, weight_decay=0)

Epoch 0: train: 1.9990402460098267
Epoch 1: train: 0.21742010116577148
Epoch 2: train: 0.14562001824378967
Epoch 3: train: 0.1316131204366684
Epoch 4: train: 0.13031113147735596
Epoch 5: train: 0.1274307370185852
Epoch 6: train: 0.12191027402877808
Epoch 7: train: 0.11977142095565796
Epoch 8: train: 0.12125454097986221
Epoch 9: train: 0.12230141460895538
Epoch 10: train: 0.11907583475112915
Epoch 11: train: 0.11844606697559357
Epoch 12: train: 0.11834661662578583
Epoch 13: train: 0.11846049129962921
Epoch 14: train: 0.11371959000825882
Epoch 15: train: 0.11063180863857269
Epoch 16: train: 0.11019828170537949
Epoch 17: train: 0.10850803554058075


KeyboardInterrupt: ignored

In [ ]:
# model=MultiLayerModel(
#     features=features_size, 
#     targets=targets_size
# )
# model = nn.Linear(in_features=features_size, out_features=targets_size)
# optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
# loss_func = nn.MSELoss()
# acc_func = RMSLELogLoss()

# for ep in range(15):
#   total_loss = 0
#   total_elements = 0
#   for fea, tar in train_dataloader:
#     pred = model(fea)
#     loss = loss_func(pred, tar)
#     loss.backward()
#     optimizer.step()
#     optimizer.zero_grad()
#     with torch.no_grad():
#       total_loss += acc_func(pred, tar)
#       total_elements += len(tar)
#   print(total_loss/total_elements)

## submit

In [21]:
dataset_submit = pd.read_csv("./data/HousePricesKaggle/test.csv")


drop_columns = 'Id'.split()
dataset_submit = dataset_submit.drop(columns=drop_columns)

select_columns = dataset_submit.dtypes[dataset_submit.dtypes != 'object'].index.tolist()
# select_columns = list(set(select_columns) - set(['SalePrice']))

dataset_submit[select_columns] = dataset_submit[select_columns].apply(lambda x: (x - x.mean()) / (x.std()))
target_columns = ['SalePrice']
dataset_submit[select_columns] = dataset_submit[select_columns].fillna(dataset_submit[select_columns].mean() / dataset_submit[select_columns].std())

OH_cols_submit = pd.DataFrame(OH_encoder.transform(dataset_submit[object_columns]))

OH_cols_submit.index = dataset_submit.index
num_X_submit = dataset_submit.drop(object_columns, axis=1)

dataset_submit = pd.concat([num_X_submit, OH_cols_submit], axis=1)

batch_size = len(dataset_submit)
submit_dataset = CustomPandasDataset(
    dataset=dataset_submit,
    target_columns=target_columns,
    infer=True
)

submit_dataloader = DataLoader(
    dataset=submit_dataset, 
    batch_size=batch_size, 
    shuffle=False,
)



In [ ]:
dataset_submit

In [23]:
with torch.no_grad():
  features, targets = next(iter(submit_dataloader))
  # print(features)
  trainer.model.eval()
  preds = trainer.model(features).cpu().numpy().reshape(-1)
  # print(preds)

  test_data_subm = pd.read_csv("./data/HousePricesKaggle/test.csv")
  test_data_subm = test_data_subm[['Id']]
  test_data_subm['SalePrice'] = preds

  test_data_subm.to_csv('./data/HousePricesKaggle/submission_my.csv', index=False)

In [ ]:
np.array([[1], [2], [3]]).reshape(-1)

array([1, 2, 3])